In [2]:
import os
import re
import cv2
import numpy as np
from os.path import isfile, join
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf


In [4]:
#reading of images in validation dataset
col_frames = os.listdir("./nus-sds-dsc2021/validation_data/validation_images")
col_frames.sort(key=lambda f: int(re.sub('\D', '', f)))

In [5]:
#reading csv file in validation dataset
df = pd.read_csv("./nus-sds-dsc2021/validation_data/validation_labels.csv")

In [6]:
#setting the standard size
#preprocess the image 
IMG_SIZE = 300

training_data = []
object_count = []
index_array = []

def create_training_data():
    
    for frame in col_frames:

            valid_image = cv2.imread("./nus-sds-dsc2021/validation_data/validation_images/" + frame)
            gray = cv2.cvtColor(valid_image, cv2.COLOR_BGR2GRAY)
            blurred = cv2.GaussianBlur(gray, (11,11) , 0)
            canny = cv2.Canny(blurred, 10 , 150, 3)
            
            dilated = cv2.dilate(canny, (1,1) , iterations = 3)
            ret2,th2 = cv2.threshold(dilated,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
            
            (cnt,hierachy ) = cv2.findContours(th2.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
            rgb = cv2.cvtColor(valid_image,cv2.COLOR_BGR2RGB)
            cv2.drawContours(rgb,cnt,-1,(0,255,0),2)

            

#             plt.imshow(rgb,cmap='gray')
            
            #locating the details of the img on the labels csv
            index = int(frame[:4])
            new_df = df.loc[df['img_id']==index]

            #resizing of img
            resized_img = cv2.resize(valid_image, (IMG_SIZE,IMG_SIZE))

            #appending to training data
            training_data.append([resized_img,index])
            object_count.append(len(cnt))
            index_array.append(index)



create_training_data()

In [7]:
#handwriting recognition attempt 1

mnist = tf.keras.datasets.mnist
(x_train,y_train),(x_test,y_test) = mnist.load_data()

x_train = tf.keras.utils.normalize(x_train,axis=1)
x_test = tf.keras.utils.normalize(x_test,axis=1)

model= tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(28,28)))
model.add(tf.keras.layers.Dense(units=128,activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(units=128,activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(units=10,activation = tf.nn.softmax))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(x_train,y_train,epochs=3)

loss, accuracy = model.evaluate(x_test,y_test)
print(accuracy)
print(loss)

model.save("digits.model")

for frame in col_frames:
    valid_image = cv2.imread("./nus-sds-dsc2021/validation_data/validation_images/" + frame)
    img = np.invert(np.array([valid_image]))
    prediction = model.predict(img)
    print("the result is probably:" + np.argmax(prediction))
    plt.imshow(valid_image,cmap = plt.cm.binary)
    plt.show()
    
    break

Epoch 1/3
1875/1875 [==============================] - 2s 896us/step - loss: 0.4635 - accuracy: 0.8693
Epoch 2/3
1875/1875 [==============================] - 2s 843us/step - loss: 0.1101 - accuracy: 0.9654
Epoch 3/3
313/313 [==============================] - 0s 588us/step - loss: 0.0959 - accuracy: 0.9698
0.9697999954223633
0.09592679142951965
INFO:tensorflow:Assets written to: digits.model/assets


ValueError: in user code:

    /Users/brennanszeto/Environments/img_recog/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1478 predict_function  *
        return step_function(self, iterator)
    /Users/brennanszeto/Environments/img_recog/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1468 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/brennanszeto/Environments/img_recog/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/brennanszeto/Environments/img_recog/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/brennanszeto/Environments/img_recog/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica

In [8]:
#handwriting recognition attempt 2 
#attempt to match the handwritten numbers based on a 40k trained dataset of numbers 0 - 9

# Read training images
# Around 42000 images
train = pd.read_csv('./nus-sds-dsc2021/validation_data/digitsCode/train.csv')
# print('shape', train.shape)


# Todo: Check indexing in pandas at https://www.shanelynn.ie/select-pandas-dataframe-rows-and-columns-using-iloc-loc-and-ix/

# Select the first 40k rows and 784 columns
features_train = train.iloc[0:40000,1:785].values
labels_train = train.iloc[0:40000,0:1].values
print('Training data shape', features_train.shape, 'Label data shape', labels_train.shape)

# Clip all values to be between 0 and 1
features_train = np.clip(features_train, 0, 1)

# Get test data, next 2K images
features_test = train.iloc[10000:12000,1:785].values
labels_test = train.iloc[10000:12000,0:1].values

# Clip all values to be between 0 and 1
features_test = np.clip(features_test, 0, 1)


from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(features_train, labels_train.ravel()) 

y_pred = clf.predict(features_test)

# get true labels from test set
y_true = labels_test.ravel()

# Print accuracy
from sklearn.metrics import accuracy_score
print('Accuracy of NB is', accuracy_score(y_true, y_pred)*100,'%')
#28 pixels in height and 28 


# Input the id of the test image
for frame in col_frames:
    valid_image = cv2.imread("./nus-sds-dsc2021/validation_data/validation_images/" + frame)
    id = int(frame[:4])

    B = np.reshape(valid_image, (28,28)) 



    # Standard pyplot command to display image
    plt.imshow(B,cmap=plt.cm.gray_r,interpolation='nearest')
    print('True label:',y_true[id], 'predicted:', y_pred[id])

Training data shape (40000, 784) Label data shape (40000, 1)


/Users/brennanszeto/Environments/img_recog/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Accuracy of NB is 37.05 %


ValueError: cannot reshape array of size 950286 into shape (28,28)

In [9]:
#exporting 
output_df = pd.DataFrame({
    'img_id': index_array,
    'acumen_count_gt': object_count,
#     'handwritten_count_gt': 
#     'anomalies_bbox_gt':
})
output_df.to_csv('NUS_hackathon.csv',index= False)